## genie Top 200

### 1. 인터넷 상에서 데이터 가져오기

In [1]:
import requests
import pandas as pd

In [2]:
# Genie Top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [4]:
# Chrome user-agent
header = {'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
req = requests.get(url, headers=header)
html = req.text
# html

In [5]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려고 하는 데이터의 태그 찾기

In [6]:
# <table class='list-wrap'>
table = soup.select_one('table.list-wrap')
trs = table.select('tr.list')
len(trs)

50

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보 추출

In [7]:
# rank, title, artist, album
tr = trs[0]
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [8]:
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [9]:
rank =int(tr.select_one('.number').get_text().split('\n')[0])
rank

1

In [12]:
# title
title = tr.select_one('.info').select_one('.title').get_text().strip()
title

'Queendom'

In [14]:
artist = tr.select_one('.info').select_one('.artist').get_text()
artist

'Red Velvet (레드벨벳)'

In [16]:
album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
album

'Queendom - The 6th Mini Album'

### 4. 한페이지에 있는 모든 데이터를 반복문으로 가져오기

In [17]:
rank_list, title_list ,artist_list, album_list = [],[],[],[]

for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').select_one('.title').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

df = pd.DataFrame({
    '순위':rank_list,
    '곡명':title_list,
    '가수명':artist_list,
    '앨범명':artist_list
})
df.tail()

,순위,곡명,가수명,앨범명
45,46,미워요,임영웅,임영웅
46,47,Timeless,SG워너비,SG워너비
47,48,사이렌 Remix (Feat. UNEDUCATED KID & Paul Blanco),호미들,호미들
48,49,At My Worst,Pink Sweat$,Pink Sweat$
49,50,내 손을 잡아,아이유 (IU),아이유 (IU)


In [18]:
df.head()

,순위,곡명,가수명,앨범명
0,1,Queendom,Red Velvet (레드벨벳),Red Velvet (레드벨벳)
1,2,신호등,이무진,이무진
2,3,낙하 (With 아이유),AKMU (악뮤),AKMU (악뮤)
3,4,바라만 본다,MSG워너비 (M.O.M),MSG워너비 (M.O.M)
4,5,Permission to Dance,방탄소년단,방탄소년단


In [19]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').select_one('.title').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    lines.append([rank,title,artist,album])

In [20]:
df = pd.DataFrame(lines,columns=['순위','곡명','가수','앨범'])
df.head()

,순위,곡명,가수,앨범
0,1,Queendom,Red Velvet (레드벨벳),Queendom - The 6th Mini Album
1,2,신호등,이무진,신호등
2,3,낙하 (With 아이유),AKMU (악뮤),NEXT EPISODE
3,4,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집
4,5,Permission to Dance,방탄소년단,Butter / Permission to Dance


### 5. 모든 페이지에서 데이터 가져오기

In [42]:
sub = 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20210817&hh=14&rtm=Y&pg='
rank_list, title_list ,artist_list, album_list = [],[],[],[]

for page in range(1,5):
    url = f'{sub}{page}'
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
        title = tr.select_one('.info').select_one('.title').get_text().strip()
        artist = tr.select_one('.info').select_one('.artist').get_text().strip()
        album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
        rank_list.append(rank)
        title_list.append(title)
        artist_list.append(artist)
        album_list.append(album)

In [43]:
df = pd.DataFrame({
    '순위':rank_list,
    '곡명':title_list,
    '가수명':artist_list,
    '앨범명':album_list
})
df.tail()

,순위,곡명,가수명,앨범명
195,196,둘 중에 골라 (Summer or Summer),효린 & 다솜,둘 중에 골라 (Summer or Summer)
196,197,광화문에서 (At Gwanghwamun),규현 (KYUHYUN),The 1st Mini Album '광화문에서 (At Gwanghwamun)'
197,198,노래방에서,장범준,장범준 3집
198,199,Rose,디오 (D.O.),공감 - The 1st Mini Album
199,200,여행,볼빨간사춘기,Red Diary Page.2
